# Guide to Using the ProSuite Python Script
This script demonstrates how to use the ProSuite Python API to create a simple quality specification (without a DDX!)
and run a here specified verification on a dataset.
It also demonstrates how to handle issues that are returned from the verification
and how to control the verification process based on the issues.
### Table of Contents:
1. [Prerequisites](#prerequisites)
2. [Connecting to the ProSuite QA Service](#connecting-to-the-prosuite-qa-service)
3. [Running Different QA Checks](#running-different-qa-checks)
    1. [Minimum Length Check](#minimum-length-check)
    2. [Segment Length Check](#segment-length-check)
    3. [Using a Shapefile for Perimeter](#using-a-shapefile-for-perimeter)
    4. [Applying an SQL Query for Filter](#applying-an-sql-query-for-filter)
4. [Loading Perimeter from a Shapefile](#loading-perimeter-from-a-shapefile)
5. [Using SQL Queries as Dataset Filters](#using-sql-queries-as-dataset-filters)
6. [Verifying a Specification](#verifying-a-specification)
7. [Handling Verification Issues](#handling-verification-issues)
8. [Generating Output and Logs](#generating-output-and-logs)

## Prerequisites
 1. **Install the ProSuite Python API (installation instructions can be found in the official ProSuite documentation).**
 2. **Make sure to run Pip from the Python distribution/environment where you need ProSuite to be installed. pip install prosuite**



In [96]:
import prosuite as ps
import os
from datetime import datetime
import time
import subprocess
import socket


 3. **Set the Path to Your Dataset**

    To use geospatial data (e.g., `.gdb` or `.sde`), follow these steps:

    Using the Sample Dataset from ProSuite:
    1. **Download the Sample Dataset**:
    - Visit the [ProSuite API Samples](https://www.dirageosystems.ch/prosuite/doc/api/samples.html) page.
     - Download and extract the dataset (e.g., to `C:/ProSuiteSampleData/`).

    2. **Adjust the Path in Your Script**:
    - After extracting, update your script to point to the correct `.gdb` file location. 



In [93]:
model_path = "C:\DIRA\SampelData\TLM_I.gdb"  # Replace this with the path to your model
output_dir = "C:\DIRA\Output"  # Replace this with the path to your output directory

model = ps.Model("TopoModel", model_path)
datasets = [ps.Dataset("TLM_FLIESSGEWAESSER_I", model),
            ps.Dataset("TLM_STRASSE_I", model)]

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\jpfra\AppData\Local\Temp\ipykernel_32204\1849169924.py:1: SyntaxWarning: invalid escape sequence '\D'
  model_path = "C:\DIRA\SampelData\TLM_I.gdb"  # Replace this with the path to your model
C:\Users\jpfra\AppData\Local\Temp\ipykernel_32204\1849169924.py:2: SyntaxWarning: invalid escape sequence '\D'
  output_dir = "C:\DIRA\Output"  # Replace this with the path to your output directory


 4. **Start the ProSuite QA Microservice**

    You have two options to start the ProSuite microservice:

    1. **Start Manually**:
    - Navigate to the folder where the `prosuite-qa-microservice.exe` is located.
    - Double-click the `.exe` file to start the service.

    2. **Start via Script**:
     - Alternatively, you can start the service programmatically using Python. Update the file path in the script to the location where your `prosuite-qa-microservice.exe` is stored:

In [97]:
# Check if the service is running
if not is_service_running():
    print("Starting the ProSuite service...")
    try:
        # Start the ProSuite service if not already running
        server_process = subprocess.Popen(r"C:\git\Dira.ProSuiteSolution\binServer_x64\Debug")
        
        # Wait for the service to start
        time.sleep(10)
        print("ProSuite service started.")
    except PermissionError as e:
        print(f"PermissionError: {e}")
    except Exception as e:
        print(f"Error starting ProSuite service: {e}")
else:
    print("Skipping service start as it's already running.")

ProSuite service is already running at localhost:5151.
Skipping service start as it's already running.


### Connecting to the ProSuite QA Service

To run quality assurance checks, you first need to establish a connection to the ProSuite QA microservice. You can either doThis is done using the `Service` class, which allows communication between your Python script and the ProSuite server. FOr usual the  default the communication channel is http://localhost:5151.

In [98]:
service = create_service('localhost', 5151)  # Adjust host and port if needed

if service:
    # Proceed with your QA checks if the service is created successfully
    print("Ready to run QA checks.")
else:
    print("Cannot proceed without connecting to the ProSuite service.")

Successfully connected to the ProSuite service on localhost:5151
Ready to run QA checks.


### Running a QA Check with a Defined Geographic Perimeter or a choosen Shapefile as perimeter

In this section, the user runs a Quality Assurance (QA) check by specifying the type of check (e.g., `min_length`, `segment_length`....) and defining a geographic perimeter ore defining the path to a shp file to use as an `EnvelopePerimeter`.

#### Key Steps:

1. **Deciding Between Shapefile Input or Manually Defined Perimeter**:
   - The user has the option to either:
     - Use a **Shapefile** to define the geographic perimeter.
     - Manually define the perimeter using coordinates (`x_min`, `y_min`, `x_max`, `y_max`).
   - **`use_shapefile`**: A boolean flag that determines whether to load a perimeter from a Shapefile (`True`) or define it directly (`False`).

2. **Choosing the Check Type**:
   - The `check_type` variable allows the user to specify which QA check to run.
     - **`min_length`**: Verifies that features meet a specified minimum length.
     - **`segment_length`**: Verifies the length of individual segments within features.
     - **`xxx`**: to be defined
     - **`xxx`**: to be defined

In [103]:
check_type = 'min_length'  # Options: 'min_length', 'segment_length', etc.

# User input
use_shapefile = True  # Set to False if the user wants to define the envelope directly
use_xml_file = False # Set to True if the user wants to verify using an XML file, otherwise set to False for QA check
shapefile_path = 'C:\\DIRA\\Output\\Output.shp'  # The user provides the shapefile path
envelope_coordinates = {
    'x_min': 2750673,
    'y_min': 1215551,
    'x_max': 2765845,
    'y_max': 1206640
}  # Define geographic perimeter

xml_file_path = 'C:\\DIRA\\specifications\\road_specification.qa.xml'  # Path to the XML file
workspace  = 'C:\\DIRA\\connection_files\\production_QA_version.gdb'  # Path to the .gdb workspace

# Determine which process to run based on the boolean flag
if use_xml_file:
    # Proceed to read and verify using the XML file
    try:
        xml_spec = prosuite.XmlSpecification(
            specification_file=xml_file_path,
            specification_name="Produktionsunterstuetzung",
            data_source_replacements=[["ProductionModel", workspace]]
        )
        
        verification_responses = service.verify(specification=xml_spec, output_dir=output_dir)
        for verification_response in verification_responses:
            print(verification_response.message)
    
    except Exception as e:
        print(f"Failed to read XML specification or run verification: {e}")

else:
    # Proceed to run the QA check
    # Determine which envelope to use
    if use_shapefile:
        envelope = load_envelope_from_shapefile(shapefile_path)
    else:
        envelope = ps.EnvelopePerimeter(**envelope_coordinates)

# Call the selected check
select_and_run_qa_check(check_type, model, datasets, output_dir, envelope)

Envelope loaded from shapefile: C:\DIRA\Output\Output.shp
Connected to ProSuite service on localhost:5151
Creating external issue file geodatabase
Status: Running
Server accessible, code running
Starting quality verification using quality specification MinimumLengthSpecification with verification tile size 5000Extent: 15172 x 8911
  X-Min: 2750673
  Y-Min: 1206640
  X-Max: 2765845
  Y-Max: 1215551

Status: Running
Server accessible, code running
Verifying quality conditions per cached tiles (container tests)
Status: Running
Server accessible, code running
  Processing tile 0 of 8: XMin: 2 750 673,00 YMin: 1 206 640,00 XMax: 2 755 673,00 YMax: 1 211 640,00
Status: Running
Server accessible, code running
  Processing tile 1 of 8: XMin: 2 755 673,00 YMin: 1 206 640,00 XMax: 2 760 673,00 YMax: 1 211 640,00
Status: Running
Server accessible, code running
  Processing tile 2 of 8: XMin: 2 760 673,00 YMin: 1 206 640,00 XMax: 2 765 673,00 YMax: 1 211 640,00
Status: Running
Server accessible, c

---------------------------------------------------------------------------------------------------------------------------------------------------------
## Definitions (No User Changes Required)

From this point onward, the code contains function definitions that are fully implemented and **do not require any modifications by the user**. These functions handle the specific details of running the Quality Assurance (QA) checks, connecting to the ProSuite service, and managing the verification process.

#### Important Information:

- The most critical sections of the code where the user needs to interact or set parameters (such as the **check type**, **model**, **datasets**, **output directory**, and **geographic envelope**) have already been explained earlier.
- The functions below handle various QA checks and internal logic, ensuring that the verification process runs smoothly without further configuration.

Feel free to explore these functions to understand their purpose, but no changes are necessary unless you're adding new types of checks.

---------------------------------------------------------------------------------------------------------------------------------------------------------

#### Function: `select_and_run_qa_check`

This function lets you run various QA checks based on the `check_type` provided.

**Key Parameters:**
- **`check_type`**: The type of QA check (e.g., `min_length`, `segment_length`).
- **`model`**: The data model.
- **`datasets`**: The datasets to check.
- **`output_dir`**: The directory for the output.
- **`envelope`**: The geographic area for the check.

**Supported QA Checks:**
1. **Minimum Length Check**: Ensures features meet a minimum length.
2. **Segment Length Check**: Ensures segments within features meet a minimum length.

**Example Usage:**
- `min_length`: Verifies features are at least 10 units long.
- `segment_length`: Verifies segments are at least 1.5 units long.

**Additional Checks:**  
You can add more checks by extending the `select_and_run_qa_check` function.



#### QA Check Functions:

1. **`qa_min_length_check(model, datasets, output_dir, envelope)`**:  
   Verifies that features meet a minimum length requirement (e.g., 10 units).

2. **`qa_segment_length_check(model, datasets, output_dir, envelope)`**:  
   Verifies that segments within features are at least 1.5 units long.

Both functions create a **specification** in ProSuite, add the necessary conditions, and run the verification process.


In [73]:
def select_and_run_qa_check(check_type, model, datasets, output_dir, envelope):
    if check_type == 'min_length':
        qa_min_length_check(model, datasets, output_dir, envelope)
    elif check_type == 'segment_length':
        qa_segment_length_check(model, datasets, output_dir, envelope)
    # Add more conditions for other checks
    #elif check_type == 'another_check':
    #    another_check(model, datasets, output_dir, envelope)
    # Add more conditions for other checks
    else:
        print(f"Unknown check type: {check_type}")

#------------------------ QA CHECKS ------------------------    
def qa_min_length_check(model, datasets, output_dir, envelope):
    specification = ps.Specification(
        name='MinimumLengthSpecification',
        description='A QA check for minimum feature length of roads and rivers.')
    
    for dataset in datasets:
        specification.add_condition(ps.Conditions.qa_min_length_0(dataset, limit=10, is3_d=False))
    
    run_verification(specification, output_dir, envelope)

# Function to create and run a "Segment Length" QA check
def qa_segment_length_check(model, datasets, output_dir, envelope):
    specification = ps.Specification(
        name='SegmentLengthSpecification',
        description='A QA check for segment length of roads and rivers.')
    
    for dataset in datasets:
        specification.add_condition(ps.Conditions.qa_segment_length_0(dataset, 1.5, False))
    
    run_verification(specification, output_dir, envelope)

#### Function: `run_verification(specification, output_dir, envelope)`

This function handles the process of connecting to the ProSuite service, running the verification with a specified quality assurance (QA) specification, and managing the verification output.

##### Key Steps:

1. **Generating a Timestamped Output Directory**:
   - The function first creates an output directory with a unique timestamp to store the verification results. This ensures that every run generates a distinct set of outputs.
   ```python
   output_dir_with_timestamp = os.path.join(output_dir, f'output_{datetime.now().strftime("%Y%m%d_%H%M%S")}')
   os.makedirs(output_dir_with_timestamp, exist_ok=True)


In [74]:
def run_verification(specification, output_dir, envelope):
    # Generate a timestamped directory for output
    output_dir_with_timestamp = os.path.join(output_dir, f'output_{datetime.now().strftime("%Y%m%d_%H%M%S")}')
    os.makedirs(output_dir_with_timestamp, exist_ok=True)
    
    # Connect to the service
    try:
        service = ps.Service(host_name='localhost', port_nr=5151)
        print(f"Connected to ProSuite service on localhost:5151")
    except Exception as e:
        print(f"Failed to connect to ProSuite service: {e}")
        return

    # Run the verification
    try:
        verification_responses = service.verify(specification=specification, output_dir=output_dir_with_timestamp, perimeter=envelope)
        
        issue_allowable = True  # This will control whether the process continues if non-allowable issues are found
        
        for verification_response in verification_responses:
            # Print general verification message
            print(verification_response.message)
            
            # Check for issues
            if len(verification_response.issues) > 0:
                for issue in verification_response.issues:
                    # Print issue details
                    print(issue.description)
                    print(issue.involved_objects)
                    print(issue.geometry)
                    print(issue.issue_code)
                    print(issue.allowable)
                    print(issue.stop_condition)

                    # If an issue is not allowable, stop the verification
                    if not issue.allowable:
                        print(f"Not allowed issue met: {issue.description} in {issue.involved_objects[0].table_name}")
                        print("Stopping verification")
                        issue_allowable = False
                        break

            # Break the outer loop if any non-allowable issue is found
            if not issue_allowable:
                break
            
            # Print the service call status
            print("Status: " + verification_response.service_call_status)

            # Handle different statuses
            if verification_response.service_call_status == "Failed":
                print("Server not accessible, check licence")
                print("Verification Response:" + verification_response.message)
            elif verification_response.service_call_status == "Running":
                print("Server accessible, code running")
            elif verification_response.service_call_status == "Cancelled":
                print("Verification cancelled")
            elif verification_response.service_call_status == "Finished":
                print("Verification finished successfully")
            else:
                print("Undefined status")

    except Exception as e:
        print(f"Verification failed: {e}")

In [100]:
def is_service_running(host='localhost', port=5151):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        try:
            s.connect((host, port))
            print(f"ProSuite service is already running at {host}:{port}.")
            return True
        except ConnectionRefusedError:
            return False

In [99]:
def create_service(host_name, port_nr):
    """
    Creates a connection to the ProSuite QA service and provides feedback on success or failure.
    
    :param host_name: The hostname of the ProSuite QA service.
    :param port_nr: The port number used by the service.
    :return: A ProSuite service object or None if the connection fails.
    """
    try:
        # Attempt to create the service instance
        service = ps.Service(host_name=host_name, port_nr=port_nr)
        print(f"Successfully connected to the ProSuite service on {host_name}:{port_nr}")
        return service
    except Exception as e:
        # Catch any exceptions and provide feedback
        print(f"Failed to connect to the ProSuite service on {host_name}:{port_nr}. Error: {e}")
        return None

In [76]:
def load_envelope_from_shapefile(shapefile_path):
    try:
        gdf = gpd.read_file(shapefile_path)
        bounds = gdf.total_bounds  # Returns (minx, miny, maxx, maxy)
        envelope = ps.EnvelopePerimeter(x_min=bounds[0], y_min=bounds[1], x_max=bounds[2], y_max=bounds[3])
        print(f"Envelope loaded from shapefile: {shapefile_path}")
        return envelope
    except Exception as e:
        print(f"Error loading shapefile: {e}")
        return None

In [75]:
#------------------------OLD CODE------------------------


simpleSpecification = ps.Specification(
    name='MinimumLengthSpecification',
    description='A very simple quality specification checking feature and segment length of roads and rivers')

for dataset in datasets:
    simpleSpecification.add_condition(ps.Conditions.qa_min_length_0(dataset, limit=10, is3_d=False))
    simpleSpecification.add_condition(ps.Conditions.qa_segment_length_0(dataset, 1.5, False))

envelope = ps.EnvelopePerimeter(x_min=2750673, y_min=1215551, x_max=2765845, y_max=1206640)

out_dir = 'C:/temp/verification_output_20' # You might want to change this to a directory that exists on your system, also make sure no Issue.gdb exists in this directory

verification_responses = service.verify(specification=simpleSpecification, output_dir=out_dir, perimeter=envelope)

# 3. Run Verification and handle issues with the Issue Object

issue_allowable = True

for verification_response in verification_responses:
    print(verification_response.message)
    if len(verification_response.issues) > 0:
        for issue in verification_response.issues:
            # Demo Prints

            print(issue.description)
            print(issue.involved_objects)
            print(issue.geometry)
            print(issue.issue_code)
            print(issue.allowable)
            print(issue.stop_condition)

            if issue.allowable is False:
                print(f"Not allowed issue met: {issue.description} in {issue.involved_objects[0].table_name}")
                print("Stopping verification")
                issue_allowable = False
                break

    if issue_allowable is False:
        break

    print("Status: "+verification_response.service_call_status)

    if(verification_response.service_call_status == "Failed" ):
        print("Server not accessible, check licence")
        print("Verification Response:"+verification_response.message)
    elif(verification_response.service_call_status == "Running"):
        print("Server accessible, code running")
    elif(verification_response.service_call_status == "Cancelled"):
        print("Cancelled")
    elif(verification_response.service_call_status == "Finished"):
        print("Server accessible, code finished running")
    else:
         print("Undifined")

The FileGdb workspace 'C:/temp/verification_output_20\Issues.gdb' already exists
Status: Running
Server accessible, code running

Status: Failed
Server not accessible, check licence
Verification Response:
